# Testes de Hipóteses

In [2]:
import pandas as pd

In [28]:
# Load gun violence dataset and gun laws dataset
gun_violence = pd.read_csv('../databases/gun_violence_clean.zip', compression='zip')
gun_law = pd.read_csv('../databases/gun_laws.zip', compression='zip')
population = pd.read_csv('../databases/population_clean.zip', compression='zip')

## Teste 1

Hipotese nula: dado que o Estado apresenta um maior número de leis que dizem respeito ao controle de porte de armas, ele apresentaria uma porcentagem de incidentes menor do que estados de menos leis.

In [36]:
gun_violence = gun_violence[['incident_id', 'state', 'n_killed', 'n_injured', 'year', 'month', 'day']]
population = population[['state', '2013', '2014', '2015', '2016', '2017', '2018']]
pop = population[5:56].reset_index().drop('index', axis=1)

incidents_state = gun_violence.groupby('state')\
                  .agg({'n_injured':'sum', 'incident_id':'count', 'n_killed':'sum'})\
                  .rename(columns={'incident_id':'number_of_incidents'})\
                  .sort_values('state')\
                  .reset_index()
states = incidents_state.join(pop, on='state', how='inner')
states

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

## Teste 2